## LU Factorization


We saw in the last section that given two matrices, $A$ and $B$, of compatible shapes, we are able to define the product matrix $C=AB$ in a useful way.  In this section we discuss the factorization of a matrix.  One might naturally ask if it is possible to start with matrix $C$ and determine the two matrix factors $A$ and $B$.  As it turns out, a useful course of action is to look for matrix factors that have a particular structure.

One such factorization, that is closely related to the elimination process, is known as the LU Factorization.  Given a matrix $A$, we will look for matrices $L$ and $U$ such that

- $LU = A$
- $L$ is a lower triangular matrix with main diagonal entries equal to 1.
- $U$ is an upper triangular matrix.

Here is a visualization of what we are seeking.


$$
\begin{equation}
A = \left[ \begin{array}{cccc} * & * & * & * \\ * & * & * & * \\ * & * & * & * \\ * & * & * & *  \end{array}\right]\hspace{1cm}
L = \left[ \begin{array}{cccc} 1 & 0 & 0 & 0 \\ * & 1 & 0 & 0 \\ * & * & 1 & 0 \\ * & * & * & 1 \end{array}\right]\hspace{1cm}
U = \left[ \begin{array}{cccc} * & * & * & * \\ 0 & * & * & * \\ 0 & 0 & * & * \\ 0 & 0 & 0 & *  \end{array}\right]\hspace{1cm}
\end{equation}
$$

Before we tackle the problem of calculating $L$ and $U$ from a known matrix $A$, let's see why such a factorization is useful.  Suppose that we have found $L$ and $U$ so that $A=LU$ and we wish to solve the system $AX=B$.  Another way to write the problem is $LUX=B$.  We can then define another unknown $Y$ by saying that $UX=Y$, and exchange the single system $AX=B$ for the following two systems.

$$
\begin{eqnarray*}
UX & = & Y\\
LY & = & B
\end{eqnarray*}
$$

While it is true that we have in fact doubled the number of equations, the two systems that we have are triangular and can be solved easily with back (or forward) substitution.  The first example shows the details for specific system.

### Example 1:  Soving a system using LU factorization

We want to solve the system of equations.

$$
\left[ \begin{array}{ccc} 3 & -1 & -2 \\ 6 & -1 & 0  \\ -3 & 5 & 20  \end{array}\right]X =
\left[ \begin{array}{c} -4 \\ -8 \\ 6  \end{array}\right]\hspace{1cm}
$$

where $X$ is an unknown $3\times 1$ vector.  Suppose we also have computed $L$ and $U$.

$$
L = \left[ \begin{array}{ccc} 1 & 0 & 0 \\ 2 & 1 & 0  \\ -1 & 4 & 1  \end{array}\right] \hspace{2cm}
U = \left[ \begin{array}{ccc} 3 & -1 & -2 \\ 0 & 1 & 4  \\ 0 & 0 & 2  \end{array}\right]
$$


In [ ]:
import numpy as np
import scipy.linalg as sla

## Use Python to check for yourself that LU = A.

Now let's write out the systems $UX=Y$ and $LY = B$.  For the sake of clarity, we leave the matrix notation aside for a moment and use the variables $x_1$, $x_2$, and $x_3$ for the entries of $X$ and the variables $y_1$, $y_2$, and $y_3$ for the entries of $Y$.


$$
\begin{eqnarray*}
x_1 \hspace{2.1cm}& = & y_1\\
2x_1 + x_2 \hspace{1.1cm}& = & y_2\\
-x_1 + 4x_2 +x_3 & = & y_3 \\
\\
3y_1 - y_2 - 2y_3 & = & -4\\
y_2 + 4y_3 & = & -8\\
2y_3 & = & 6
\end{eqnarray*}
$$

Now the solution is a matter of substitution.  The last equation tells us $y_3$.  From there we work backwards to find $y_2$ and $y_1$.  Then we go the first three equations to determine the $x$ values in a similar way, starting this time with the very first equation and working our way down.

### Elementary matrices

In order to understand how we can construct the LU factorization through elimination, it helpful to see that the steps of elimination can be carried out by multiplication with special matrices called **elementary matrices**.  Elementary matrices are the result of applying either a $\texttt{RowScale}$ or $\texttt{RowAdd}$ operation to the identity matrix of compatible shape.  (*Remember that rearranging the rows is only necessary if a 0 arises in a pivot position.  We will address row swaps shortly.*)

For an example, let's apply one of these operations to a $4\times 4$ identity matrix.

In [ ]:
I = np.eye(4)
# Function to perform row addition
def RowAdd(A, i, j, scalar):
    """
    Perform the operation R_i = R_i + scalar * R_j
    A: Matrix (NumPy array)
    i: Row to be updated (zero-based index)
    j: Row to be added (zero-based index)
    scalar: Scalar to multiply the row j by before adding to row i
    """
    A[i, :] = A[i, :] + scalar * A[j, :]
    return A

# Perform the row operation: Row 1 = Row 1 + (-3) * Row 2
E = RowAdd(I.copy(), 1, 2, -3)
print(I,'\n')
print(E)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

[[ 1.  0.  0.  0.]
 [ 0.  1. -3.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


The $E$ that we get is the result of adding -3 times the first row of $I$ to the third row of $I$.  The interesting property of the elementary matrix $E$ is that if we multiply another matrix $A$ by $E$, the result will be a the matrix we would get by applying the same row operation to $A$.

In [ ]:
A=np.array([[1,2,0,-1],[-1,1,-1,4],[2,13,-4,9],[-2,5,-3,13]])
print(A,'\n')
print(E@A)

[[ 1  2  0 -1]
 [-1  1 -1  4]
 [ 2 13 -4  9]
 [-2  5 -3 13]] 

[[  1.   2.   0.  -1.]
 [ -7. -38.  11. -23.]
 [  2.  13.  -4.   9.]
 [ -2.   5.  -3.  13.]]


### Example 2:  Finding an LU factorization

We can now carry out the elimination by applying a sequence of elementary matrices $E_1$, $E_2$, $E_3$,...to $A$.  Let's see how it works for the matrix above.

In [ ]:
A=np.array([[1,2,0,-1],[-1,1,-1,4],[2,13,-4,9],[-2,5,-3,13]])
I = np.eye(4)
# Perform the first set of row operations
E1 = RowAdd(I.copy(), 0, 1, 1)   # Row 0 = Row 0 + 1 * Row 1
E2 = RowAdd(I.copy(), 0, 2, -2)  # Row 0 = Row 0 - 2 * Row 2
E3 = RowAdd(I.copy(), 0, 3, 2)   # Row 0 = Row 0 + 2 * Row 3
# Print result of applying the row operations to A
print(E3 @ E2 @ E1 @ A, '\n')
# Perform the second set of row operations
E4 = RowAdd(I.copy(), 1, 2, -3)  # Row 1 = Row 1 - 3 * Row 2
E5 = RowAdd(I.copy(), 1, 3, -3)  # Row 1 = Row 1 - 3 * Row 3
# Print result of applying the additional row operations to A
print(E5 @ E4 @ E3 @ E2 @ E1 @ A)

[[ -8. -13.   1.  11.]
 [ -1.   1.  -1.   4.]
 [  2.  13.  -4.   9.]
 [ -2.   5.  -3.  13.]] 

[[ -8. -13.   1.  11.]
 [ -1. -53.  20. -62.]
 [  2.  13.  -4.   9.]
 [ -2.   5.  -3.  13.]]


After using $\texttt{RowAdd}$ to create zeros in the appropriate spaces, we now have the $U$ factor.  Writing out the matrix multiplication in symbols it looks like this.

$$
\begin{equation}
E_5E_4E_3E_2E_1A = U
\end{equation}
$$

Note that the order of the multiplications cannot be changed.  $E_1$ should be the first to multiply $A$, then $E_2$, and so on.  Now let us manipulate the symbols a bit based on the properties of inverse matrices.

$$
\begin{eqnarray}
A &=& (E_5E_4E_3E_2E_1)^{-1}U  \\
A &=& E_1^{-1}E_2^{-1}E_3^{-1}E_4^{-1}E_5^{-1}U  
\end{eqnarray}
$$

It must be that $L = E_1^{-1}E_2^{-1}E_3^{-1}E_4^{-1}E_5^{-1}$.  The fact that this product of inverse elementary matrices has the correct form to be $L$ is not at all clear.  Let's make the following two observations.

- Each of the inverse elementary matrices has a simple lower triangular structure.  In fact, the matrix $E_3^{-1}$ is also an elementary matrix.  It is the elementary matrix that undoes the row operation represented by $E_3$!  Multiplication by $E_3$ adds 2 times the first row to the last row.  Multiplication by $E_3^{-1}$ adds -2 times the first row to the last row.

In [ ]:
print(E3,'\n')
print(sla.inv(E3))

[[1. 0. 0. 2.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

[[ 1. -0.  0. -2.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1. -0.]
 [ 0.  0.  0.  1.]]


- Multiplying two lower triangular matrices together produces a lower triangular matrix.  Look at any example and try to figure out why.

In [ ]:
L1 = np.array([[1,0,0,0],[-1,1,0,0],[2,3,1,0],[-2,3,0,1]])
L2 = np.array([[1,0,0,0],[2,1,0,0],[-5,4,1,0],[4,4,1,1]])
print(L1,'\n')
print(L2,'\n')
print(L1@L2)

[[ 1  0  0  0]
 [-1  1  0  0]
 [ 2  3  1  0]
 [-2  3  0  1]] 

[[ 1  0  0  0]
 [ 2  1  0  0]
 [-5  4  1  0]
 [ 4  4  1  1]] 

[[1 0 0 0]
 [1 1 0 0]
 [3 7 1 0]
 [8 7 1 1]]


These two facts together tell us that the matrix $E_1^{-1}E_2^{-1}E_3^{-1}E_4^{-1}E_5^{-1}$ has the correct structure to be the $L$ factor.  What is even more convenient is that when we multiply these inverse elementary matrices together, the nonzero  entries in the lower triangular portions do not change.

In [ ]:
print(sla.inv(E5),'\n')
print(sla.inv(E4)@sla.inv(E5),'\n')
print(sla.inv(E3)@sla.inv(E4)@sla.inv(E5))

[[ 1. -0.  0. -0.]
 [ 0.  1. -0.  3.]
 [ 0.  0.  1. -0.]
 [ 0.  0.  0.  1.]] 

[[1. 0. 0. 0.]
 [0. 1. 3. 3.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

[[ 1.  0.  0. -2.]
 [ 0.  1.  3.  3.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


The nonzero lower triangular entries in $E_3^{-1}E_4^{-1}E_5^{-1}$ are the same as the corresponding entries of $E_3^{-1}$, $E_4^{-1}$, and $E_5^{-1}$.  This means that the entries in $L$ are just the scale factors used in our application of $\texttt{RowAdd}$, multiplied by -1.  Now that we understand how these elementary matrices combine to produce $L$, we don't actually need to construct them.  We can just compute $L$ as we do the row operations by keeping track of the scale factors.  

In [ ]:
L = np.array([[1,0,0,0],[-1,1,0,0],[2,3,1,-0],[-2,3,0,1]])
U = np.array([[1,2,0,-1],[0,3,-1,3],[0,0,-1,2],[0,0,0,2]])
print("L:",'\n',L,'\n',sep='')
print("U:",'\n',U,'\n',sep='')
print("LU:",'\n',L@U,sep='')

L:
[[ 1  0  0  0]
 [-1  1  0  0]
 [ 2  3  1  0]
 [-2  3  0  1]]

U:
[[ 1  2  0 -1]
 [ 0  3 -1  3]
 [ 0  0 -1  2]
 [ 0  0  0  2]]

LU:
[[ 1  2  0 -1]
 [-1  1 -1  4]
 [ 2 13 -4  9]
 [-2  5 -3 13]]


### Permutation matrices

As we have seen in the previous section, it is sometimes necessary to rearrange the rows of a matrix when performing elimination.  This row operation can also be done by multiplying the matrix with an elementary matrix.  Let's build a matrix $P$ that performs an exchange of rows 2 and 3 in a $4\times 4$ matrix.  Again, we can do this by performing the same row operation on the identity matrix.  

In [ ]:
C = np.random.randint(-6,6,size=(4,4))
I = np.eye(4)

# Custom RowSwap function
def RowSwap(A, i, j):
    """
    Swap rows i and j in matrix A.
    A: Matrix (NumPy array)
    i: First row index to be swapped (zero-based index)
    j: Second row index to be swapped (zero-based index)
    """
    A[[i, j], :] = A[[j, i], :]  # Swap rows i and j
    return A

# Create a random 4x4 matrix C with values between -6 and 6
C = np.random.randint(-6, 6, size=(4, 4))
# Identity matrix I
I = np.eye(4)
# Perform row swap operation on the identity matrix
P = RowSwap(I.copy(), 1, 2)  # Swap rows 1 and 2 in the identity matrix
# Print the results
print("C:", '\n', C, '\n', sep='')
print("P (after RowSwap):", '\n', P, '\n', sep='')
print("PC (after multiplying P and C):", '\n', P @ C, sep='')


C:
[[-5 -3  4  0]
 [-1  4  1 -3]
 [ 3  2  5  5]
 [-4 -3  3 -4]]

P (after RowSwap):
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]

PC (after multiplying P and C):
[[-5. -3.  4.  0.]
 [ 3.  2.  5.  5.]
 [-1.  4.  1. -3.]
 [-4. -3.  3. -4.]]


When the row operation is a row swap, it is common to refer to the corresponding elementary matrix as a **permutation matrix**, and use the letter $P$ to represent it.  We will follow this convention.  It is also useful to note that an elementary permutation matrix is its own inverse since the operation of swapping two rows can be reversed by performing the exact same operation.  We can check that $PP=I$, which means that $P=P^{-1}$.

In [ ]:
print(P@P)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


Note that if a permutation represents more than a single row exchange, then its inverse must represent those row exhanges applied in the reverse order.

### Example 3:  Factoring with row swaps

In order to understand how the row swaps are incorporated into the factorization, it is most helpful to see an example.  In this $4\times 4$ example, we will use our functions to carry out the elimination and build the corresponding elementary matrices along the way.  For the $\texttt{RowAdd}$ operations, we will label the elementary matrix with an $L$, and for the $\texttt{RowSwap}$ operations we will use the label $P$.

In [ ]:
B = np.array([[1,2,-1,-1],[4,8,-4,2],[1,1,1,2],[3,3,4,4]])
print(B)

[[ 1  2 -1 -1]
 [ 4  8 -4  2]
 [ 1  1  1  2]
 [ 3  3  4  4]]


In [ ]:
# Perform the first set of row operations on B and I
B = RowAdd(B.copy(), 0, 1, -4)  # Row 0 = Row 0 - 4 * Row 1
L1 = RowAdd(I.copy(), 0, 1, -4)  # Same operation on the identity matrix

# Perform the second set of row operations
B = RowAdd(B.copy(), 0, 2, -1)  # Row 0 = Row 0 - 1 * Row 2
L2 = RowAdd(I.copy(), 0, 2, -1)  # Same operation on the identity matrix

# Perform the third set of row operations
B = RowAdd(B.copy(), 0, 3, -3)  # Row 0 = Row 0 - 3 * Row 3
L3 = RowAdd(I.copy(), 0, 3, -3)  # Same operation on the identity matrix

# Print the final matrix B after row operations
print("Matrix B after row operations:")
print(B)

Matrix B after row operations:
[[-25 -40   2 -23]
 [  4   8  -4   2]
 [  1   1   1   2]
 [  3   3   4   4]]


In [ ]:
# Perform row swap on B and I
B = RowSwap(B.copy(), 1, 2)  # Swap Row 1 and Row 2 in matrix B
P1 = RowSwap(I.copy(), 1, 2)  # Swap Row 1 and Row 2 in identity matrix I
# Print the final matrix B after the row swap
print("Matrix B after row swap:")
print(B)
# Optionally, print the transformation matrix P1
print("\nTransformation matrix P1 (after RowSwap on identity):")
print(P1)

Matrix B after row swap:
[[-25 -40   2 -23]
 [  1   1   1   2]
 [  4   8  -4   2]
 [  3   3   4   4]]

Transformation matrix P1 (after RowSwap on identity):
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [ ]:
# Perform row addition on B and I
B = RowAdd(B.copy(), 1, 3, -3)  # Row 1 = Row 1 + (-3) * Row 3 in matrix B
L4 = RowAdd(I.copy(), 1, 3, -3)  # Same operation on identity matrix I
# Print the final matrix B after the row addition
print("Matrix B after row addition:")
print(B)
# Optionally, print the transformation matrix L4
print("\nTransformation matrix L4 (after RowAdd on identity):")
print(L4)

Matrix B after row addition:
[[-25 -40   2 -23]
 [ -8  -8 -11 -10]
 [  4   8  -4   2]
 [  3   3   4   4]]

Transformation matrix L4 (after RowAdd on identity):
[[ 1.  0.  0.  0.]
 [ 0.  1.  0. -3.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [ ]:
# Perform row swap on B and I (swap row 2 and row 3)
B = RowSwap(B.copy(), 2, 3)  # Swap Row 2 and Row 3 in matrix B
P2 = RowSwap(I.copy(), 2, 3)  # Swap Row 2 and Row 3 in identity matrix I
# Print the final matrix B after the row swap
print("Matrix B after row swap:")
print(B)
# Optionally, print the transformation matrix P2
print("\nTransformation matrix P2 (after RowSwap on identity):")
print(P2)

Matrix B after row swap:
[[-25 -40   2 -23]
 [ -8  -8 -11 -10]
 [  3   3   4   4]
 [  4   8  -4   2]]

Transformation matrix P2 (after RowSwap on identity):
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In terms of matrix multiplication, we have carried out the matrix product $P_2L_4P_1L_3L_2L_1B = U$, as we can check.

In [ ]:
B = np.array([[1,2,-1,-1],[4,8,-4,2],[1,1,1,2],[3,3,4,4]])
U = P2@L4@P1@L3@L2@L1@B
print(U)

[[-25. -40.   2. -23.]
 [ -8.  -8. -11. -10.]
 [  3.   3.   4.   4.]
 [  4.   8.  -4.   2.]]


As we see with a calculation in the next cell, the inverse matrix $(P_2L_4P_1L_3L_2L_1)^{-1}$ does not have the correct lower triangular structure to be the $L$ factor.    In fact there are no matrices $L$ and $U$ with the correct triangular structure such that $B=LU$ for this particular matrix $B$.

In [ ]:
possible_L = sla.inv(P2@L4@P1@L3@L2@L1)
print(possible_L)

[[ 1.  1.  6.  4.]
 [ 0.  0. -0.  1.]
 [ 0.  1.  3.  0.]
 [ 0.  0.  1.  0.]]


Although this matrix does not have the correct structure, we might notice that it is only a matter of rearranging the rows to produce a lower triangular matrix.  In fact, the row swaps that are needed here are *exactly the same* as those used in the elimination process.  

In [ ]:
L = P2@P1@possible_L
print(L)

[[1. 1. 6. 4.]
 [0. 1. 3. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


Now let's put all of this together to make the factorization of $B$.  Elimination gives us that $B = (P_2L_4P_1L_3L_2L_1)^{-1}U$, but the matrix $(P_2L_4P_1L_3L_2L_1)^{-1}$ is not lower triangular.  We can produce a lower triangular factor by multiplying by the permutation matrices that produce the required row swaps.

$$
P_2P_1B = P_2P_1(P_2L_4P_1L_3L_2L_1)^{-1}U
$$

We will label $P_2P_1(P_2L_4P_1L_3L_2L_1)^{-1}$ as $L$, since it now has the correct structure.  The end result is that $B=PLU$ where $P=(P_2P_1)^{-1}$.  To compute the inverse of the permutation matrix, we can simply apply the row swaps in the reverse order, so that $P=P_1P_2$.

In [ ]:
P = P1@P2
print("P\n",P,'\n',sep='')
print("L\n",L,'\n',sep='')
print("U\n",U,'\n',sep='')
print("PLU\n",P@L@U,sep='')

## Note the optional argument sep='' to print eliminates the space usually included between printed items

P
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]

L
[[1. 1. 6. 4.]
 [0. 1. 3. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

U
[[-25. -40.   2. -23.]
 [ -8.  -8. -11. -10.]
 [  3.   3.   4.   4.]
 [  4.   8.  -4.   2.]]

PLU
[[ 1.  2. -1. -1.]
 [ 4.  8. -4.  2.]
 [ 1.  1.  1.  2.]
 [ 3.  3.  4.  4.]]


The important point here is that if row swaps are used during elimination, a permutation matrix will be required in the factorization in order for $L$ to have the desired triangular structure.  Therefore in general we expect that $B=PLU$ where $P$ represents all the row swaps that occur during elimination.  

Another way to arrive at this result is to realize that if we performed the required row swaps *before* starting the elimination process, they would not interfere with the structure of $L$.  Let's give it a try!

In [ ]:
# Initialize matrix B
B = np.array([[1, 2, -1, -1],
              [4, 8, -4, 2],
              [1, 1, 1, 2],
              [3, 3, 4, 4]])
# Perform row swap operations
B = RowSwap(B.copy(), 1, 2)  # Swap Row 1 and Row 2
B = RowSwap(B.copy(), 2, 3)  # Swap Row 2 and Row 3
# Print the final matrix B after the row swaps
print("Matrix B after row swaps:")
print(B)

Matrix B after row swaps:
[[ 1  2 -1 -1]
 [ 1  1  1  2]
 [ 3  3  4  4]
 [ 4  8 -4  2]]


In [ ]:
# Perform the row addition operations on B and I
B = RowAdd(B.copy(), 0, 1, -1)  # Row 0 = Row 0 + (-1) * Row 1
L1 = RowAdd(I.copy(), 0, 1, -1)  # Same operation on the identity matrix
B = RowAdd(B.copy(), 0, 2, -3)  # Row 0 = Row 0 + (-3) * Row 2
L2 = RowAdd(I.copy(), 0, 2, -3)  # Same operation on the identity matrix
B = RowAdd(B.copy(), 0, 3, -4)  # Row 0 = Row 0 + (-4) * Row 3
L3 = RowAdd(I.copy(), 0, 3, -4)  # Same operation on the identity matrix
# Print the final matrix B after the row additions
print("Matrix B after row additions:")
print(B)
# Optionally, print the transformation matrices L1, L2, L3
print("\nTransformation matrix L1 (after RowAdd on identity):")
print(L1)
print("\nTransformation matrix L2 (after RowAdd on identity):")
print(L2)
print("\nTransformation matrix L3 (after RowAdd on identity):")
print(L3)

Matrix B after row additions:
[[-25 -40   2 -23]
 [  1   1   1   2]
 [  3   3   4   4]
 [  4   8  -4   2]]

Transformation matrix L1 (after RowAdd on identity):
[[ 1. -1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]

Transformation matrix L2 (after RowAdd on identity):
[[ 1.  0. -3.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]

Transformation matrix L3 (after RowAdd on identity):
[[ 1.  0.  0. -4.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [ ]:
# Perform the row addition operation
B = RowAdd(B.copy(), 1, 2, -3)  # Row 1 = Row 1 + (-3) * Row 2
L4 = RowAdd(I.copy(), 1, 2, -3)  # Same operation on the identity matrix
# Print the final matrix B after the row addition
print("Matrix B after row addition:")
print(B)
# Optionally, print the transformation matrix L4
print("\nTransformation matrix L4 (after RowAdd on identity):")
print(L4)

Matrix B after row addition:
[[-25 -40   2 -23]
 [ -8  -8 -11 -10]
 [  3   3   4   4]
 [  4   8  -4   2]]

Transformation matrix L4 (after RowAdd on identity):
[[ 1.  0.  0.  0.]
 [ 0.  1. -3.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


The process has given us $L_4L_3L_2L_1P_2P_1B=U$. Now $(L_4L_3L_2L_1)^{-1}$ has the correct structure, and is the same matrix $L$ that we produced in the previous calculation.

In [ ]:
L = sla.inv(L4@L3@L2@L1)
print(L)

[[ 1.  1.  6.  4.]
 [ 0.  1.  3. -0.]
 [ 0.  0.  1. -0.]
 [ 0.  0.  0.  1.]]


### PLU factorization with SciPy

The SciPy $\texttt{linalg}$ function for finding $PLU$ factorizations is called $\texttt{lu}$.  This function accepts the array to be factored as an argument, and returns three arrays representing $P$, $L$, and $U$, in that order.  To store these arrays for later use, we need to provide three names that will be assigned to the output of $\texttt{lu}$.  

In [ ]:

B = np.array([[1,2,-1,-1],[4,8,-4,2],[1,1,1,2],[3,3,4,4]])
P,L,U = sla.lu(B)

print("P\n",P,'\n',sep='')
print("L\n",L,'\n',sep='')
print("U\n",U,'\n',sep='')
print("PLU\n",P@L@U,sep='')

P
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

L
[[ 1.          0.          0.          0.        ]
 [ 0.75        1.          0.          0.        ]
 [ 0.25        0.33333333  1.          0.        ]
 [ 0.25       -0.         -0.          1.        ]]

U
[[ 4.          8.         -4.          2.        ]
 [ 0.         -3.          7.          2.5       ]
 [ 0.          0.         -0.33333333  0.66666667]
 [ 0.          0.          0.         -1.5       ]]

PLU
[[ 1.  2. -1. -1.]
 [ 4.  8. -4.  2.]
 [ 1.  1.  1.  2.]
 [ 3.  3.  4.  4.]]


We can see that the $\texttt{lu}$ function produced matrices $P$, $L$, and $U$ such that $B=PLU$, but the factors are different than those we found using our own row operations.  Both factorizations are correct as we can see.  It is important to realize that the factorization of $B$ into $PLU$ is not unique since there is a choice to be made in which rows get swapped.  Instead of only requiring that pivot elements are non-zero, the SciPy function chooses row swaps using a more advanced method in order to minimize potential roundoff error.

SciPy can also be used to solve a system $AX=B$ by using the $PLU$ factorization of $A$ together with back and forward substitution.  To do this we use $\texttt{lu_factor}$ to factor and $\texttt{lu_solve}$ to carry out the substitutions.

In [ ]:
A = np.array([[1,2,-1,-1],[4,8,-4,2],[1,1,1,2],[3,3,4,4]])
X_true = np.array([[1],[0],[1],[0]])
B = A@X_true

## lu_factor returns two objects which store the information needed to reconstruct the factorization
## We store it all in a single object called 'factorization', which can then be passed to lu_solve
factorization = sla.lu_factor(A)
X_computed = sla.lu_solve(factorization,B)

print("Computed solution X:\n",X_computed,'\n',sep='')


Computed solution X:
[[ 1.]
 [-0.]
 [ 1.]
 [-0.]]



The results returned from $\texttt{lu_factor}$ are not the same as those returned from $\texttt{lu}$.  The underlying factorization is the same, but the information is compressed into a more efficient format.  

### Exercises

**Exercise 1:** Solve $ AX = B $ using $ A = LU $ and the $L$, $U$, and $B$ given below.  Compute $LUX$ to verify your answer.

$$
\begin{equation}
A = \left[ \begin{array}{ccc} 5 & 2 & 1 \\ 5 & 3 & 0 \\ -5 & -2 & -4  \end{array}\right] \hspace{2cm}
B = \left[ \begin{array}{c} 4 \\ 7 \\ 8  \end{array}\right] \hspace{2cm}
L = \left[ \begin{array}{ccc} 1 & 0 & 0 \\ 1 & 1 & 0  \\ -1 & 0 & 1  \end{array}\right] \hspace{2cm}
U = \left[ \begin{array}{ccc} 5 & 2 & 1 \\ 0 & 1 & -1  \\ 0 & 0 & 3  \end{array}\right]
\end{equation}
$$

In [ ]:
## Code solution here.

**Exercise 2:** Solve $ AX = B $ using $ A = LU $ and the $L$, $U$, and $B$ given below.  Compute $LUX$ to verify your answer.

$$
\begin{equation}
L = \left[ \begin{array}{ccc} 1 & 0 & 0 \\ -1 & 1 & 0 \\ 0 & -1 & 1  \end{array}\right] \hspace{2cm}
U = \left[ \begin{array}{ccc} 1 & -1 & 0 \\ 0 & 1 & -1  \\ 0 & 0 & 1  \end{array}\right] \hspace{2cm}
B = \left[ \begin{array}{c} 2 \\ -3 \\ 4  \end{array}\right]
\end{equation}
$$

In [ ]:
## Code solution here.

**Exercise 3:** Write a function called $\texttt{ForwardSubstitution}$ that will solve a lower triangular system $LY=B$.  It will be helpful to go back and look at the code for $\texttt{BackSubstitution}$.

In [ ]:
## Code solution here.

**Exercise 4:** Let $A$, $B$, and $C$ be the following matrices:

  $$
\begin{equation}
A = \left[ \begin{array}{ccc} 1 & 2 & 4 \\ 2 & 1 & 3 \\ 1 & 0 & 2  \end{array}\right] \hspace{2cm}
B = \left[ \begin{array}{ccc} 1 & 2 & 4 \\ 2 & 1 & 3  \\ 2 & 2 & 6  \end{array}\right] \hspace{2cm}
C = \left[ \begin{array}{ccc} 1 & 2 & 4 \\ 0 & -1 & -3  \\ 2 & 2 & 6  \end{array}\right]
\end{equation}
$$

$(a)$ Find an elementary matrix $E$ such that $EA = B$.  Verify with a computation.

$(b)$ Find an elementary matrix $F$ such that $ FB = C$.  Verify with a computation.
    

In [ ]:
## Code solution here.

**Exercise 5:** Consider the following $3\times 3$  matrix :

$$
\begin{equation}
A = \left[ \begin{array}{rrr}  2 & 1 & 1\\  6 & 4 & 5  \\ 4 & 1 & 3 \end{array} \right]
\end{equation}
$$

$(a)$ Find **elementary matrices** $E_1$, $E_2$, and $E_3$ such that $ E_3E_2E_1A = U $ where $U$ is an upper triangular matrix.

$(b)$ Find $L$ using the inverses of $E_1$, $E_2$, $E_3$, and verify that $  A = LU $.

In [ ]:
## Code Solution here

**Exercise 6:** Compute the $LDU$ factorization of the following matrix and verify that $A = LDU$.

$$
\begin{equation}
A = \left[ \begin{array}{rrr}  1 & 1 & 1\\  3 & 5 & 6  \\ -2 & 2 & 7 \end{array} \right]
\end{equation}
$$


In [ ]:
## Code Solution here

**Exercise 7:** Find $P, L,$ and $U$ such that $PA = LU$.  Following the discussion in this section, $P$ should be a permutation matrix, $L$ should be a lower triangular matrix with ones long the main diagonal, and $U$ should be an upper triangular matrix.

$$
\begin{equation}
A = \left[ \begin{array}{rrr}  1 & 3 & 2\\  -2 & -6 & 1  \\ 2 & 5 & 7 \end{array} \right]
\end{equation}
$$
    

In [ ]:
## Code Solution here

**Exercise 8:** Use SciPy to compute the $PLU$ factorization of a $3\times 3$ matrix.  Replicate the results using the row operations functions as done above.

In [ ]:
## Code Solution here